In [1]:
_ = !pip install opendatasets

In [2]:
import pandas as pd
import numpy as np
import opendatasets as od
import glob
import os
import re

### Import Dataset

In [3]:
#Download datasets from Kaggle
od.download("https://www.kaggle.com/grouplens/movielens-20m-dataset")
#24bf23baff0e58c26b0fbff70c0803c1

Skipping, found downloaded files in "./movielens-20m-dataset" (use force=True to force download)


In [4]:
#Import data
variables = locals()
path = 'movielens-20m-dataset/'
for file in os.listdir(path):
    name = str(file.rsplit('.', 1)[0])
    variables["df_{0}".format(name)] = pd.read_csv(path + file)

In [5]:
%whos DataFrame

Variable           Type         Data/Info
-----------------------------------------
df_genome_scores   DataFrame              movieId  tagId <...>1709768 rows x 3 columns]
df_genome_tags     DataFrame          tagId           tag<...>\n[1128 rows x 2 columns]
df_link            DataFrame           movieId   imdbId  <...>n[27278 rows x 3 columns]
df_movie           DataFrame           movieId           <...>n[27278 rows x 3 columns]
df_rating          DataFrame              userId  movieId<...>0000263 rows x 4 columns]
df_tag             DataFrame            userId  movieId  <...>[465564 rows x 4 columns]


In [6]:
df_rating.shape

(20000263, 4)

In [7]:
df_rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [8]:
df_movie.shape

(27278, 3)

In [9]:
print('Rating de', len(df_rating.movieId.unique()), 'films,', 'de un total de', len(df_rating.userId.unique()), 'usuarios.')

Rating de 26744 films, de un total de 138493 usuarios.


#### Create Target

In [10]:
df_rating['fl_high'] = np.where(df_rating['rating'] >= 4, 1,0)

#### Dataset Raitings

- Rolling average rating

In [11]:
user_MA = df_rating[['userId','movieId','rating','timestamp']].sort_values(by='timestamp', ascending=True)
user_MA['lag_rating'] = user_MA.groupby('userId')['rating'].shift(1)
user_MA = user_MA.sort_values(by=['userId','timestamp'], ascending=True)
user_MA['mov_avg_rating'] = user_MA.groupby('userId')['lag_rating'].transform(lambda x: x.rolling(5, 1).mean())
user_MA['mov_avg_rating'] = user_MA['mov_avg_rating'].fillna(0)
#user_MA.head()

#### Split Data
- Realizo el split de train y test por fecha, distibuyendo 80/20 ya que considero que la variable tiempo es importante y una buena opción para evitar la fuga de datos.

- Fecha Rating

In [12]:
## Para evitar la fuga de datos al generar variables realizo el split de train-val-test
#de la ventana de tienpo de Ene-1995 a Mar-2015 verificamos que no hay entrada de datos durante 11 meses desde Feb-1995 a Dic-1995
df_rating['date'] = pd.to_datetime(df_rating['timestamp'])

In [13]:
df_rating['yyyymm'] = df_rating['date'].dt.to_period('M')

In [14]:
end_date = df_rating['date'].max()
start_date = df_rating['date'].min()
(end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)


242

In [15]:
user_MA = user_MA[['userId','movieId','lag_rating','mov_avg_rating']]
df_rating_feat = df_rating.merge(user_MA, left_on = ['userId','movieId'], right_on=['userId','movieId'], how='left')

In [16]:
df_rating_feat[df_rating_feat['userId']==1]

,userId,movieId,rating,timestamp,fl_high,date,yyyymm,lag_rating,mov_avg_rating
0,1,2,3.5,2005-04-02 23:53:47,0,2005-04-02 23:53:47,2005-04,4.0,3.7
1,1,29,3.5,2005-04-02 23:31:16,0,2005-04-02 23:31:16,2005-04,3.5,3.7
2,1,32,3.5,2005-04-02 23:33:39,0,2005-04-02 23:33:39,2005-04,3.5,3.7
3,1,47,3.5,2005-04-02 23:32:07,0,2005-04-02 23:32:07,2005-04,4.0,3.9
4,1,50,3.5,2005-04-02 23:29:40,0,2005-04-02 23:29:40,2005-04,3.0,3.4
...,...,...,...,...,...,...,...,...,...
170,1,8507,5.0,2004-09-10 03:13:47,1,2004-09-10 03:13:47,2004-09,3.5,3.6
171,1,8636,4.5,2005-04-02 23:44:53,1,2005-04-02 23:44:53,2005-04,3.5,3.8
172,1,8690,3.5,2005-04-02 23:33:15,0,2005-04-02 23:33:15,2005-04,4.0,3.8
173,1,8961,4.0,2005-04-02 23:47:09,1,2005-04-02 23:47:09,2005-04,3.5,3.5


In [17]:
#Split train-val-test por fecha de rating
start_train = '1995-01'
end_train = '2007-12'
df_train = df_rating_feat[(df_rating_feat.yyyymm >= start_train) & (df_rating_feat.yyyymm <= end_train)]
df_test = df_rating_feat[(df_rating_feat.yyyymm > end_train)]

In [18]:
print('Train:' , df_train.shape[0], 'size: %', round(df_train.shape[0] / df_rating.shape[0]* 100.0,2)) 
print('Test:' , df_test.shape[0], 'size: %', round(df_test.shape[0] / df_rating.shape[0]* 100.0,2)) 

Train: 14063903 size: % 70.32
Test: 5936360 size: % 29.68


### Features

- Se generaron las siguientes features en base a usuario y peliculas

    * Time Moving Average rating por usuario
    * Rating promedio por usuario
    * Media y Desviacion de rating por usuario
    * Rating de película anterior
    * antiguedad en meses por usuario desde su primer rating
    * Cantidad de peliculas votadas por usuario
    * Variables dummy de generos de cada película
    * Año Pelicula
    * Rating promedio por pelicula
    * Media y Desviacion de rating por pelicula
    * Popularidad de la película


### Generate features

In [19]:
def generate_features (df):
    df_movie_rating = df.groupby(['movieId'])['rating'].agg(movies_pop = 'count', avg_movie_rating='mean', std_movie_rating='std', median_movie_rating='median')
    df_rating_user = df.groupby(['userId'])['rating'].agg(movies_views = 'count', avg_user_rating='mean',std_user_rating='std', median_user_rating='median')
    df_user_time = df.groupby(['userId'])['timestamp'].agg(start = 'min', last='max')
    df_user_time[['start','last']] = df_user_time[['start','last']].apply(pd.to_datetime)
    df_user_time['antiguedad'] = (df_user_time['last'] - df_user_time['start']).dt.days
    df_user_time = df_user_time.drop(columns=['start','last'], axis=1)
    df_rating_user = df_rating_user.merge(df_user_time, left_index=True, right_index=True, how='left')
    
    return df_movie_rating, df_rating_user

In [20]:
df_movie_rating, df_rating_user = generate_features(df_train)

### Dataset Users

In [21]:
# dataset origen en la funcion generate_features
df_rating_user.head()

,movies_views,avg_user_rating,std_user_rating,median_user_rating,antiguedad
userId,,,,,
1,175,3.742857,0.382284,4.0,204
2,61,4.000000,1.064581,4.0,0
3,187,4.122995,0.910427,4.0,3
4,28,3.571429,0.790151,4.0,0
5,66,4.272727,0.969464,5.0,1


#### Dataset Peliculas

- Año de pelicula

In [23]:
#extraemos el año del film del título, observo que no siempre se encuentran entre parentesis y en algunos casos no son los unicos numeron entre parentesis por lo que ajusto el siguiente RegeX
df_movie['year'] = df_movie.title.str.extract(r"\((\d{4})", expand=True)[0]
df_movie[df_movie.isna().any(axis=1)].shape
#Tenemos 19 peliculas que no contamos con el dato de año

(19, 4)

- Generos

In [27]:
generos = list(df_movie['genres'].str.split('|'))
flat_list = [item for sublist in generos for item in sublist]
flat_list.remove('(no genres listed)')
len(set(flat_list))

20

In [28]:
movie_oh_genres = df_movie['genres'].str.get_dummies('|')

In [29]:
movie_oh_genres[movie_oh_genres['(no genres listed)']==1].shape

(246, 20)

In [30]:
movie_oh_genres = movie_oh_genres.drop(columns=['(no genres listed)'],  axis=1)

In [44]:
df_movies_oh = df_movie.merge(movie_oh_genres, left_index=True, right_index=True, how='left')
df_movies_oh = df_movies_oh.merge(df_movie_rating, left_on='movieId', right_index=True, how='left')
df_movies_oh.head()

,movieId,title,genres,year,Action,Adventure,Animation,Children,Comedy,Crime,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,movies_pop,avg_movie_rating,std_movie_rating,median_movie_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,0,1,1,1,1,0,...,0,0,0,0,0,0,37694.0,3.946344,0.893341,4.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,0,1,0,1,0,0,...,0,0,0,0,0,0,17276.0,3.233329,0.960672,3.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,0,0,0,0,1,0,...,0,1,0,0,0,0,11847.0,3.161475,1.009853,3.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,0,0,0,0,1,0,...,0,1,0,0,0,0,2668.0,2.869753,1.089626,3.0
4,5,Father of the Bride Part II (1995),Comedy,1995,0,0,0,0,1,0,...,0,0,0,0,0,0,10745.0,3.103211,0.955402,3.0


In [45]:
#Dataset de peliculas
df_movies_oh = df_movies_oh.drop(columns=['title','genres'], axis=1)

In [33]:
df_train.head()

,userId,movieId,rating,timestamp,fl_high,date,yyyymm,lag_rating,mov_avg_rating
0,1,2,3.5,2005-04-02 23:53:47,0,2005-04-02 23:53:47,2005-04,4.0,3.7
1,1,29,3.5,2005-04-02 23:31:16,0,2005-04-02 23:31:16,2005-04,3.5,3.7
2,1,32,3.5,2005-04-02 23:33:39,0,2005-04-02 23:33:39,2005-04,3.5,3.7
3,1,47,3.5,2005-04-02 23:32:07,0,2005-04-02 23:32:07,2005-04,4.0,3.9
4,1,50,3.5,2005-04-02 23:29:40,0,2005-04-02 23:29:40,2005-04,3.0,3.4


In [34]:
df_rating_user.head()

,movies_views,avg_user_rating,std_user_rating,median_user_rating,antiguedad
userId,,,,,
1,175,3.742857,0.382284,4.0,204
2,61,4.000000,1.064581,4.0,0
3,187,4.122995,0.910427,4.0,3
4,28,3.571429,0.790151,4.0,0
5,66,4.272727,0.969464,5.0,1


In [35]:
df_movies_oh.head()

,movieId,year,movies_pop,avg_movie_rating,std_movie_rating,median_movie_rating
0,1,1995,37694.0,3.946344,0.893341,4.0
1,2,1995,17276.0,3.233329,0.960672,3.0
2,3,1995,11847.0,3.161475,1.009853,3.0
3,4,1995,2668.0,2.869753,1.089626,3.0
4,5,1995,10745.0,3.103211,0.955402,3.0


### Dataset final

In [53]:
def generate_dataset(df):
    movielens_data = df.drop(columns=['rating', 'timestamp', 'date', 'yyyymm'], axis = 1)
    movielens_data = movielens_data.merge(df_rating_user, left_on=['userId'], right_index=True, how='left')
    movielens_data = movielens_data.merge(df_movies_oh, left_on=['movieId'], right_on=['movieId'], how='left')
    #hadle miss values on test_data with median
    col_null = movielens_data.columns[movielens_data.isna().any()].tolist()
    for col in col_null:
        movielens_data[col].fillna(movielens_data[col].median(),inplace=True)
    movielens_data = movielens_data.set_index(['userId', 'movieId'])
    return movielens_data

In [54]:
train_data = generate_dataset(df_train)
test_data = generate_dataset(df_test)

In [50]:
train_data.shape

(14063903, 32)

In [55]:
test_data.shape

(5936360, 32)

In [49]:
test_data.head()

fl_high  lag_rating  mov_avg_rating  movies_views  \
userId movieId                                                      
11     1              1         5.0             3.2         480.0   
       10             0         4.0             3.9         480.0   
       19             0         3.5             4.3         480.0   
       32             1         3.0             4.1         480.0   
       39             1         4.0             4.2         480.0   

                avg_user_rating  std_user_rating  median_user_rating  \
userId movieId                                                         
11     1               3.459611         0.877361                 3.5   
       10              3.459611         0.877361                 3.5   
       19              3.459611         0.877361                 3.5   
       32              3.459611         0.877361                 3.5   
       39              3.459611         0.877361                 3.5   

                antiguedad  year  Action  ...  Mystery  Romance  Sci-Fi  \
userId movieId                            ...                             
11     1             730.0  1995       0  ...        0        0       0   
       10            730.0  1995       1  ...        0        0       0   
       19            730.0  1995       0  ...        0        0       0   
       32            730.0  1995       0  ...        1        0       1   
       39            730.0  1995       0  ...        0        1       0   

                Thriller  War  Western  movies_pop  avg_movie_rating  \
userId movieId                                                         
11     1               0    0        0     37694.0          3.946344   
       10              1    0        0     24757.0          3.429091   
       19              0    0        0     16658.0          2.566485   
       32              1    0        0     34883.0          3.874394   
       39              0    0        0     22687.0          3.444616   

                std_movie_rating  median_movie_rating  
userId movieId                                         
11     1                0.893341                  4.0  
       10               0.856181                  3.0  
       19               1.140531                  3.0  
       32               0.891318                  4.0  
       39               0.966184                  3.5  

[5 rows x 32 columns]

### Export

In [57]:
train_data['year'] = train_data['year'].astype(int)
test_data['year'] = test_data['year'].astype(int)

In [58]:
train_data.to_parquet('data/train_data.parquet', engine='pyarrow')
test_data.to_parquet('data/test_data.parquet', engine='pyarrow')

### Other Features

* Por cuestion de tiempos no pude, pero me hubiese gustado intentar probar realizar una multiplicación de vectores con el rating promedio por usuario de todos los generos y el vector boleano de llos generos de la película, dividido por la cantidad de generos de las peliculas. Creo que esta es una manera en que podemos captar la relacion de las preferencias del usuario en cuanto a generos específico.
* Si la fecha fuera realmente el momento en el que vio y puntuó la pelicula, podriamos utilizar esa información aplicando una transformacion de seno, coseno sobre la variable tiempo, ya que podría afectar la puntuación de un usuario.
* Por un tema de tiempos no consideré la variable de tags ya que a mis consideración para su correcta implementación es necesario eliminar el efecto de las etiquetas creadas libremente por el usuario, mediante un proceso de mapeo, las etiquetas originales que comparten el contexto común se agrupan en una nueva etiqueta asociada con una puntuación compuesta. Más específicamente, se realiza un paso de limpieza que incluye lematización y eliminación de palabras vacías y caracteres no alfabéticos para generar una forma apropiada de genoma de etiqueta sin procesar. Luego aplicar word2vec para agruparlos. 
* Analizar la posibilidad de incluir data externa como la base de Imdb relacionado por Id, en el cual podriamos considerar los protagonistas de las pelicula, el director, presupuesto, entre otras.